In [ ]:
!pip install playwright
!playwright install


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 21.1 MB/s eta 0:00:00
172.5 MiB [] 0% 0.0s172.5 MiB [] 0% 7.8s172.5 MiB [] 0% 3.3s172.5 MiB [] 1% 2.5s172.5 MiB [] 2% 2.1s172.5 MiB [] 4% 1.9s172.5 MiB [] 5% 1.8s172.5 MiB [] 5% 1.9s172.5 MiB [] 6% 1.9s172.5 MiB [] 7% 1.8s172.5 MiB [] 9% 1.7s172.5 MiB [] 11% 1.6s172.5 MiB [] 12% 1.5s172.5 MiB [] 13% 1.5s172.5 MiB [] 14% 1.4s172.5 MiB [] 16% 1.4s172.5 MiB [] 17% 1.3s172.5 MiB [] 19% 1.2s172.5 MiB [] 20% 1.2s172.5 MiB [] 21% 1.2s172.5 MiB [] 22% 1.1s172.5 MiB [] 24% 1.1s172.5 MiB [] 25% 1.1s172.5 MiB [] 26% 1.1s172.5 MiB [] 27% 1.1s172.5 MiB [] 29% 1.0s172.5 MiB [] 31% 1.0s172.5 MiB [] 33% 0.9s172.5 MiB [] 35% 0.9s172.5 MiB [] 37% 0.8s172.5 MiB [] 37% 0.9s172.5 MiB [] 37% 1.0s172.5 MiB [] 38% 1.0s172.5 MiB [] 38% 1.1s172.5 MiB [] 39% 1.1s172.5 MiB [] 40% 1.0s172.5 MiB [] 41% 1.0s172.5 MiB [] 42% 1.0s172.5 MiB [] 44% 0.9s172.5 MiB [] 46% 0.9s172.5 MiB [] 47% 0.8s172.5 MiB [] 49% 0.8s172.5 MiB [] 50% 0.8s172.5 MiB [] 51% 0.8s172.5 Mi

In [ ]:
!pip install nest_asyncio

In [ ]:
# Step 1: Install required packages if not already done
!pip install playwright nest_asyncio
!playwright install

# Step 2: Import and prepare environment
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import json

nest_asyncio.apply()

# Step 3: Define the async scraping function
async def scrape_apple_threads(query="trade ins", start_page=1):
    base_url = "https://discussions.apple.com"
    search_url_base = f"{base_url}/search?source=aml&origin=asc_serp&q={query.replace(' ', '+')}&page="

    scraped_threads = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        # page = await browser.new_page()

        for pageno in range(start_page, start_page + 1):
            page = await browser.new_page()
            search_url = search_url_base + str(pageno)
            print(f"🔎 Visiting search page {pageno}")
            await page.goto(search_url)
            await page.wait_for_selector("a[href^='/thread/']", timeout=90000) # Increased timeout

            links = await page.locator("a[href^='/thread/']").all()
            ## filter the links that doesn't have '?' in them
            filtered_links = set([link for link in links if '?' not in await link.get_attribute("href")])

            for link in filtered_links:
                href = await link.get_attribute("href")
                full_url = base_url + href
                title = await link.inner_text()

                # Visit the thread page
                thread_page = await browser.new_page()
                try:
                    await thread_page.goto(full_url)
                    await thread_page.wait_for_timeout(60000)

                    try:
                        question = await thread_page.locator("div.question-title").inner_text()
                    except:
                        question = title

                    try:
                        responses = await thread_page.locator("div.content-post-body-content").all_inner_texts()
                        description = responses[0] if responses else "No description found"
                        answer = responses[1:] if len(responses) > 1 else "No reply found"
                    except:
                        description = "No description found"
                        answer = "No reply found"


                    scraped_threads.append({
                        "title": title.strip(),
                        "url": full_url,
                        "question": question.strip(),
                        "description": description.strip(),
                        "answer": answer
                    })

                    print(f"✅ Scraped: {title.strip()}")

                except Exception as e:
                    print(f"❌ Error scraping {full_url}: {str(e)}")

                await thread_page.close()

            await page.close()

        await browser.close()

    # Save to JSON file
    with open(f"apple_tradein_threads_{start_page}.json", "w") as f:
        json.dump(scraped_threads, f, indent=2)

    print(f"\n🎉 Done! Scraped {len(scraped_threads)} threads.")

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import json

nest_asyncio.apply()

async def get_discussion_links(end_page, query="trade ins", start_page=1) :
    base_url = "https://discussions.apple.com"
    search_url_base = f"{base_url}/search?source=aml&origin=asc_serp&q={query.replace(' ', '+')}&page="

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        # page = await browser.new_page()

        discussion_links = []

        for pageno in range(start_page, end_page + 1):
            page = await browser.new_page()
            search_url = search_url_base + str(pageno)
            print(f"🔎 Visiting search page {pageno}")
            await page.goto(search_url)
            await page.wait_for_selector("a[href^='/thread/']", timeout=90000) # Increased timeout

            links = await page.locator("a[href^='/thread/']").all()
            ## filter the links that doesn't have '?' in them
            filtered_links = set([str(link.get_attribute("href")) for link in links if '?' not in await link.get_attribute("href")])

            with open("discussion_links.links", "a+") as f :
                f.writelines(filtered_links)


Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [ ]:
import asyncio

async def main_scraping_task(pages):
    tasks = []
    for i in range(1, pages+1):
        tasks.append(scrape_apple_threads(query="trade ins", start_page=i))
    await asyncio.gather(*tasks)

# Run the main scraping task in the background
loop = asyncio.get_event_loop()
loop.run_until_complete(main_scraping_task(pages=5))

✅ Scraped: Why does Apple issue Gift Cards on Trade Ins instead of crediting card used to purchase?
✅ Scraped: Trade-In Multiple Devices
✅ Scraped: Phone trade ins
✅ Scraped: I'm wondering if I can trade in an iPad 2nd Generation and an older Macbook ?
🔎 Visiting search page 1
🔎 Visiting search page 5
🔎 Visiting search page 2
🔎 Visiting search page 4
🔎 Visiting search page 3
✅ Scraped: I have a question? When you trade in phone for a new one do you have time to sync the phones or do you have to give it to apple immediately? Any response would be appreciated. thanks...
✅ Scraped: Phone trade in
✅ Scraped: during covid can you still do trade ins in store
✅ Scraped: Does apple offer trade ins for devices from other companies for store credit?
✅ Scraped: Trade ins
✅ Scraped: Can you trade in more than one phone when buying a new one?
✅ Scraped: Trade ins
✅ Scraped: How much would it be next year to trade in the iPhone 11 for the iPhone 11Pro?
✅ Scraped: I have an Apple Watch series 2. Will

KeyboardInterrupt: 

In [ ]:
import json
import glob

output_file = "apple_tradein_combined_faq.json"
json_files = glob.glob("apple_tradein_threads_*.json")

all_data = []
for file in json_files:
    with open(file, 'r') as f:
        data = json.load(f)
        all_data.extend(data)

with open(output_file, 'w') as f:
    json.dump(all_data, f, indent=2)

print(f"Combined {len(all_data)} entries into {output_file}")

Combined 222 entries into apple_tradein_combined_faq.json


In [ ]:
import json
import pandas as pd
import re
import unicodedata

# Load data from data.json
try:
    with open('apple_tradein_combined_faq.json', 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    print("Error: apple_tradein_combined_faq.json not found. Please make sure the file exists.")
    data = []

# Convert to DataFrame for easier handling
df = pd.DataFrame(data)

# --- Cleaning Steps ---

# 1. Handle Missing Values (Example: Fill missing strings with "N/A", numbers with 0)
# Ensure 'answer' is a list of strings and join them for CSV
def format_answer_for_csv(answer):
    if isinstance(answer, list):
        # Join list elements into a single string for CSV
        return ". ".join([str(item) if item is not None else "" for item in answer]).replace("..", ".")
    elif answer is None:
        return "" # Replace None with an empty string
    else:
        return str(answer) # Ensure other values are strings

if 'answer' in df.columns:
    df['answer'] = df['answer'].apply(format_answer_for_csv)


df.dropna(inplace=True)


# 2. Remove Duplicates (Based on a relevant column, e.g., 'question')
df.drop_duplicates(subset=['question'], inplace=True)
df.drop_duplicates(subset=['answer'], inplace=True)


# 3. Basic Text Cleaning (Example: Remove leading/trailing whitespace, extra spaces, and normalize unicode)
def clean_text(text):
    if isinstance(text, str):
        # Remove leading/trailing whitespace
        text = text.strip()
        # Remove extra spaces within text
        text = re.sub(r'\n+', ' ', text)
        text = re.sub(r'\s+', ' ', text)

        # Normalize unicode characters
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

for col in ['title', 'url', 'question', 'description']: # Exclude 'answer' from this cleaning as it's a list
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

# Save the cleaned data to a new CSV file
output_csv_file = 'apple_tradein_threads.csv'
df.to_csv(output_csv_file, index=False)


print(f"Data cleaning complete. Cleaned data saved to {output_csv_file}")

Data cleaning complete. Cleaned data saved to apple_tradein_threads.csv


In [ ]:
! pip install jsonlines chromadb transformers tqdm


In [ ]:
import jsonlines
import chromadb
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Load data from JSONL file
data_path = "/apple_tradein_combined_faq.jsonl"
data = []
with jsonlines.open("/content/cleaned_data.jsonl") as reader:
    for entry in reader:
        if "question" in entry and "answer" in entry:
            data.append({
                "id": str(hash(entry["question"])),
                "question": entry["question"],
                "description": entry.get("description", ""),
                "answer": ". ".join(entry["answer"]).replace("..", ".")
            })

# Initialize embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings
questions = [item["question"] for item in data]
embeddings = embedding_model.encode(questions, show_progress_bar=True)

# Initialize ChromaDB
client = chromadb.Client()
collection = client.get_or_create_collection(name="apple_tradein_faq")

# Ingest data into ChromaDB
for i, item in enumerate(data):
    collection.add(
        documents=[item["question"]],
        embeddings=[embeddings[i]],
        metadatas=[{"answer": item["answer"]}],
        ids=[item["id"]]
    )

# Run a test query
test_query = "Can I trade in my iPhone online?"
query_embedding = embedding_model.encode([test_query])[0]

# Retrieve top 3 relevant entries
results = collection.query(query_embeddings=[query_embedding], n_results=3)
results_output = []
for i in range(len(results["documents"][0])):
    results_output.append({
        "Question": results['documents'][0][i],
        "Answer": results['metadatas'][0][i]['answer']
    })

import pandas as pd
df = pd.DataFrame(results_output)
print(df)


In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.6: Fast Qwen2 patching. Transformers: 4.55.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["question"]
    inputs       = examples["description"]
    outputs      = examples["answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Ensure output is a string by joining list elements if necessary
        if isinstance(output, list):
            output = ". ".join(output).replace("..", ".")
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("csv", data_files = "apple_tradein_threads.csv")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'], # Select the 'train' split
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 108 | Num Epochs = 5 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.843400
2,2.254400
3,1.652100
4,1.926400
5,2.055300
6,1.551700
7,1.961800
8,1.667800
9,1.571100
10,2.001300


In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "apple take trade ins to and give credit a new watch?", # instruction
        "Does apple take trade ins  to a new watch?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
apple take trade ins to and give credit a new watch?

### Input:
Does apple take trade ins  to a new watch?

### Response:
Yes.  See the link below.  You can use the trade-in value towards the cost of a new watch.  You can also sell it yourself and use the proceeds towards the cost of a new watch.  Apple Trade In - Apple. Yes.  See the link below.  You can use the trade-in value towards the cost of a new watch.  You can also sell it yourself and use the proceeds towards the cost of a new watch.  Apple Trade In - Apple<|endoftext|>
